In [34]:
import pandas as pd
import numpy as np
import pickle
from gensim.models import KeyedVectors
import torch
import ast

In [121]:
# Embedding size (Google's Word2Vec model uses embedding size of 300)
E = 300

# As hidden dimension we'll use 2E
H = 2*E

# Number of epochs
epochs = 2

# Longest sentence size
S = 120

In [8]:
# load vocab and convert to list
vocab = pickle.load(open("data/vocab_simple.p", "rb"))

In [10]:
inf_vocab

{nan: 13721,
 'mussel': 29545,
 'safest': 4869,
 'pronounced': 12950,
 'expositions': 43209,
 'atmos': 39364,
 'delegate': 13972,
 'careful': 29204,
 'graben': 41848,
 'bachao': 7660,
 'metastasize': 23152,
 'vilify': 36661,
 'prude': 32530,
 'chemical': 1797,
 'tohelp': 25932,
 'ukrainian': 6959,
 'smegma': 39122,
 'uncomfortable': 10623,
 'actresss': 44115,
 'containership': 44972,
 'tracing': 5411,
 'oozing': 27738,
 'undulate': 32405,
 'evaporated': 18241,
 'deserved': 22406,
 'dy': 26453,
 'hich': 35748,
 'fashionable': 7882,
 'dispiriting': 45140,
 'quarry': 34824,
 'zapatista': 18762,
 'littered': 30008,
 'offshoot': 45769,
 'desperately': 13770,
 'rigging': 25170,
 'confidence': 797,
 'appearing': 9824,
 'uncharacteristically': 31286,
 'sheriff': 20992,
 'doodly': 43974,
 'unstable': 8298,
 'servant': 2954,
 'referrals': 17116,
 'overthinking': 3456,
 '5th': 4701,
 'tiss': 13270,
 'satyam': 16200,
 'passerby': 12080,
 'caliph': 24597,
 'bloodhound': 30452,
 'anwers': 37769,
 'f

In [7]:
# generate inverse vocab
inf_vocab = {x : y for y, x in enumerate(vocab)} 

In [11]:
# size of vocab
V = len(vocab)

In [44]:
# load Google Word2Vec set (takes long, only execute if necessary)
# model = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [52]:
# empty embedding matrix
emb_matrix = np.zeros((V,E))

# loop over vocab
for idx, word in enumerate(vocab):
    if word not in model.vocab:
        continue
        
    emb_matrix[idx] = model[word]
    
# store embedding matrix
pickle.dump(emb_matrix, open("data/embedding_matrix.p", "wb" ))

In [12]:
# load embedding matrix
emb_matrix = pickle.load(open("data/embedding_matrix.p", "rb"))

In [97]:
# Create LSTM model
lstm = torch.nn.LSTM(E, H)
linear = torch.nn.Linear(H, V)
loss_fn = torch.nn.MSELoss()
softmax = torch.nn.functional.softmax

In [22]:
# Create embedding layer
embeddings = torch.Tensor(emb_matrix)
embedding = torch.nn.Embedding(embeddings.size(0), embeddings.size(1))
embedding.weight = torch.nn.Parameter(embeddings)

In [28]:
tokenized = pd.read_csv("data/tokenized_questions.csv")

In [70]:
def zero_hidden():
    return (torch.zeros(1, 1, H),
            torch.zeros(1, 1, H))

In [105]:
# Optimizer
optimizer = torch.optim.Adam(list(lstm.parameters()), lr=0.001)

In [95]:
def exponent_neg_manhattan_distance(left, right):
    return torch.exp(-torch.sum(torch.abs(left-right)))
#     return torch.abs(left-right)

In [119]:
x = [1,2,3]
embed = [0] * 10
start = 10 -len(x)
embed[start:10] = x
len(embed)

10

In [125]:
# Epochs
for _ in range(epochs):

    i = 0
    
    # Loop over dataset
    for _, s in tokenized.iterrows():
        
        # Create empty paddings
        q1 = [0] * S
        q2 = [0] * S
        
        # Fetch raw questions (literal_eval because I fucked up the formatting in PD)
        q1_raw = ast.literal_eval(s['question1'])
        q2_raw = ast.literal_eval(s['question2'])
        
        # Add raw questions to the end of the padding
        q1[(S - len(q1_raw)):S] = q1_raw
        q2[(S - len(q2_raw)):S] = q2_raw
        
        # Format questions 
        q1 = torch.tensor(q1)
        q2 = torch.tensor(q2)
        
        # Embed
        q1_input = embedding(q1).unsqueeze(1)
        q2_input = embedding(q2).unsqueeze(1)
        
        # LSTM
        q1_output, _ = lstm(q1_input, zero_hidden())
        q2_output, _ = lstm(q2_input, zero_hidden())
        
        # What should the value be?
        wanted = torch.tensor(float(s['is_duplicate']))
        
        # Get the result
        result = exponent_neg_manhattan_distance(q1_output, q2_output)

        # Calculate loss
        loss = loss_fn(result, wanted)
        
        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
            
    print('Loss: {:6.4f}'.format(loss.item()))

tensor(0., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(3.1561e-32, grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(0.9989, grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(3.0175e-36, grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(2.3683e-10, grad_fn=<MseLossBackward>)
tensor(0.9877, grad_fn=<MseLossBackward>)
tensor(1.2139e-39, grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(1.9959e-37, grad_fn=<MseLossBackward>)
tensor(1.2987e-07, grad_fn=<MseLossBackward>)
tensor(8.5479e-44, grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(5.4701e-23, grad_fn=<MseLossBackward>)
tensor(1.1087e-36, grad_fn=<MseLossBackward>)
tensor(8